In [1]:
import mip

class BranchAndBoundBinary:
    def __init__(self):
        self.qtd_variaveis = None
        self.qtd_restrições = None
        self.funcao_objetivo = None
        self.restricoes = None
        self.limite_superior = float('inf')
        self.limite_inferior = float('-inf')
        self.modelo_geral = mip.Model(sense = mip.MAXIMIZE, solver_name= mip.CBC)
        self.variaveis = None
        self.fila = []

    def __leitor_txt(self, arquivo):
        try:
            with open(arquivo, 'r') as arquivo:
                conteudo = arquivo.readlines()
                conteudo = [x.split() for x in conteudo]
                conteudo = [[int(y) for y in x] for x in conteudo]
                self.qtd_variaveis = conteudo[0][0]
                self.qtd_restrições = conteudo[0][1]
                self.funcao_objetivo = conteudo[1]
                self.restricoes = conteudo[2:]
        except FileNotFoundError:
            print('Arquivo não encontrado')

    def __criacao_do_modelo(self):
        x = [self.modelo_geral.add_var(var_type=mip.CONTINUOUS, name = f"x_{i}") for i in range(self.qtd_variaveis)]
        self.modelo_geral.objective = mip.xsum(self.funcao_objetivo[i] * x[i] for i in range(self.qtd_variaveis))
        # Adicionando as restrições
        for i in range(self.qtd_restrições):
            self.modelo_geral += mip.xsum(self.restricoes[i][j] * x[j] for j in range(self.qtd_variaveis)) <= self.restricoes[i][-1]
        # Resolvendo o modelo
        self.modelo_geral.optimize()
        # Adicionando os resultados das variáveis na lista solução
        self.variaveis = x

    def __is_binary(self, solucao):
        lista_nao_binarias = []
        for i in range(len(solucao)):
            if solucao[i] != 0.0 and solucao[i] != 1:
                lista_nao_binarias.append(solucao[i])
        return lista_nao_binarias

    def __var_ramification(self, solucao):
        # variáveis da solução que não são binarias
        variaveis_nao_binarias = self.__is_binary(solucao)
        # das variáveis não inteiras, qual é a que tem casa decimal mais próxima de 0.5
        variavel_mais_proxima_de_05 = [abs(round(x - int(x), 4) - 0.5) for x in variaveis_nao_binarias]
        index = variavel_mais_proxima_de_05.index(min(variavel_mais_proxima_de_05))
        # recupera o valor da variável na lista de inteiras
        var = variaveis_nao_binarias[index]
        # recupera o índice da variável na lista de variáveis
        index = solucao.index(var)

        return index

    # salva modelo em arquivo lp, e mostra o conteúdo
    def save(self, model, filename):
        model.write(filename) # salva modelo em arquivo
        with open(filename, "r") as f: # lê e exibe conteúdo do arquivo
            print(f.read())

    def resolver_modelo(self, arquivo):
        self.__leitor_txt(arquivo)
        self.__criacao_do_modelo()
        # Define o limite superior
        self.limite_superior = self.modelo_geral.objective_value
        self.fila.append(self.modelo_geral)
        melhor_solucao = None
        while self.fila:
            modelo = self.fila.pop(0)
            status = modelo.optimize()
            if status != mip.OptimizationStatus.OPTIMAL:
                continue

            # verifica se a solução é binaria
            solucao = [x.x for x in modelo.vars]
            if len(self.__is_binary(solucao)) == 0:
                if melhor_solucao == None:
                    melhor_solucao = modelo
                else:
                    if modelo.objective_value > melhor_solucao.objective_value:
                        melhor_solucao = modelo
        
            # se não for binario, ramifica
            else:
                # verifica se a solução é binária e se é melhor que o limite inferior, atualiza o limite inferior e poda o nó
                if len(self.__is_binary(solucao)) == 0 and modelo.objective_value > self.limite_inferior:
                    self.limite_inferior = modelo.objective_value
                else:
                    # se não for binário e for melhor que o limite inferior, ramifica
                    if modelo.objective_value > self.limite_inferior:
                        p1, p2 = self.sub_problema(modelo)
                    else:
                        # independente de ser binário ou não, se não for melhor que o limite inferior, poda
                        print("modelo podado")
                        continue
                self.fila.append(p1)
                self.fila.append(p2)
        return melhor_solucao
        

    def sub_problema(self, modelo, indice_inteiro = None):
        # cria uma cópia do modelo
        modelo_p2 = modelo.copy()
        modelo_p1 = modelo.copy()

        var = [x for x in modelo.vars]
        var_values = [x.x for x in modelo.vars]
        
        indice_variavel_ramificar = self.__var_ramification(var_values)
    
        # adiciona a restrição de ramificação
        modelo_p1 += var[indice_variavel_ramificar] == 0
        modelo_p2 += var[indice_variavel_ramificar] == 1

        return modelo_p1, modelo_p2


In [2]:
branch_binary = BranchAndBoundBinary()
solucao = branch_binary.resolver_modelo('teste1.txt')
branch_binary.save(solucao, 'modelo_1.lp')
print("Função Objetivo: ", solucao.objective_value)	
print("Solução: ", [x.x for x in solucao.vars])

\Problem name: 

Minimize
OBJROW: -2 x_0 -10 x_1 -8 x_2 -7 x_3 -10 x_4 -10 x_5 -6 x_6
Subject To
constr(0):  5 x_0 + 7 x_1 + 8 x_2 + x_3 + 7 x_4 + 5 x_5 + 6 x_6 <= 20
constr(1):  x_0 + 6 x_1 + 4 x_2 + 9 x_3 + 10 x_4 + 6 x_5 + 10 x_6 <= 30
constr(2):  4 x_0 + 4 x_1 + 4 x_2 + x_3 + 5 x_4 + 5 x_5 + 10 x_6 <= 40
constr(3):  3 x_0 + 10 x_1 + 8 x_2 + x_3 + 3 x_4 + 3 x_5 + 8 x_6 <= 30
constr(4):  10 x_0 + 8 x_1 + 9 x_2 + 9 x_3 + 7 x_4 + 6 x_5 + 10 x_6 <= 20
constr(5):  6 x_0 + 6 x_1 + 3 x_2 + 6 x_3 + 3 x_4 + 7 x_5 + 2 x_6 <= 80
constr(6):  7 x_0 + 10 x_1 + 7 x_2 + 8 x_3 + 7 x_4 + 8 x_5 + 7 x_6 <= 100
constr(7):  9 x_0 + 8 x_1 + x_2 + x_3 + 8 x_4 + 10 x_5 + 2 x_6 <= 90
constr(8):  x_0 + 5 x_1 + 3 x_2 + 10 x_3 + 2 x_4 + 4 x_5 + 9 x_6 <= 70
constr(9):  9 x_0 + 6 x_1 + x_2 + 4 x_3 + 7 x_4 + 5 x_5 + 10 x_6 <= 60
constr(10):  5 x_0 + 7 x_1 + 4 x_2 + 4 x_3 + 3 x_4 + 4 x_5 + 10 x_6 <= 40
constr(11):  x_5 = 0
constr(12):  x_4 = 1
constr(13):  x_1 = 1
constr(14):  x_2 = 0
constr(15):  x_3 = 0
constr(16

In [3]:
branch_binary = BranchAndBoundBinary()
solucao = branch_binary.resolver_modelo('teste2.txt')
branch_binary.save(solucao, 'modelo_2.lp')
print("Função Objetivo: ", solucao.objective_value)
print("Solução: ", [x.x for x in solucao.vars])

\Problem name: 

Minimize
OBJROW: -7 x_0 -7 x_1 -7 x_2 -5 x_3 -8 x_4 -8 x_5 -9 x_6 -10 x_7 -7 x_8
Subject To
constr(0):  x_0 + 3 x_1 + x_2 + 3 x_3 + 3 x_4 + 7 x_5 + 2 x_6 + x_7 + 4 x_8 <= 80
constr(1):  7 x_0 + 6 x_1 + 10 x_2 + x_3 + 7 x_4 + 2 x_5 + 2 x_6 + 7 x_7 + x_8 <= 90
constr(2):  3 x_0 + 2 x_1 + x_2 + 3 x_3 + 3 x_4 + 2 x_5 + x_6 + 6 x_7 + 5 x_8 <= 10
constr(3):  10 x_0 + 8 x_1 + 3 x_2 + 6 x_3 + 10 x_4 + 7 x_5 + 3 x_6 + 10 x_7 + 4 x_8 <= 30
constr(4):  2 x_0 + 8 x_1 + 6 x_2 + 5 x_3 + 6 x_4 + 6 x_5 + 9 x_6 + 7 x_7 + 2 x_8 <= 80
constr(5):  3 x_0 + 10 x_1 + x_2 + 9 x_3 + 2 x_4 + 7 x_5 + 7 x_6 + 9 x_7 + 10 x_8 <= 90
constr(6):  x_0 + 7 x_1 + 10 x_2 + 10 x_3 + 5 x_4 + 2 x_5 + 9 x_6 + 10 x_7 + 2 x_8 <= 20
constr(7):  10 x_0 + 3 x_1 + 2 x_2 + 3 x_3 + 10 x_4 + 2 x_5 + x_6 + 9 x_7 + 7 x_8 <= 10
constr(8):  2 x_0 + x_1 + 7 x_2 + 10 x_3 + x_4 + x_5 + 2 x_6 + x_7 + x_8 <= 50
constr(9):  x_5 = 1
constr(10):  x_6 = 1
constr(11):  x_8 = 1
Bounds
End

Função Objetivo:  24.0
Solução:  [0.0, 0.0,

In [4]:
branch_binary = BranchAndBoundBinary()
solucao = branch_binary.resolver_modelo('teste3.txt')
branch_binary.save(solucao, 'modelo_3.lp')
print("Função Objetivo: ", solucao.objective_value)
print("Solução: ", [x.x for x in solucao.vars])

\Problem name: 

Minimize
OBJROW: -7 x_0 -9 x_1 -10 x_2 -3 x_3 -6 x_4 - x_5 -9 x_6 -8 x_7 -8 x_8
Subject To
constr(0):  2 x_0 + x_1 + 9 x_2 + 6 x_3 + 3 x_4 + 6 x_5 + 10 x_6 + 9 x_7 + x_8 <= 60
constr(1):  8 x_0 + 6 x_1 + 6 x_2 + 5 x_3 + 2 x_4 + 2 x_5 + 4 x_6 + 3 x_7 + 6 x_8 <= 80
constr(2):  8 x_0 + x_1 + 3 x_2 + 7 x_3 + x_4 + 4 x_5 + 8 x_6 + 3 x_7 + 4 x_8 <= 30
constr(3):  6 x_0 + 3 x_1 + 9 x_2 + 5 x_3 + 9 x_4 + 6 x_5 + 9 x_6 + 9 x_7 + 6 x_8 <= 40
constr(4):  10 x_0 + 8 x_1 + 8 x_2 + 7 x_3 + 10 x_4 + 10 x_5 + 9 x_6 + 9 x_7 + 3 x_8 <= 20
constr(5):  10 x_0 + 10 x_1 + 10 x_2 + 9 x_3 + 10 x_4 + x_5 + 8 x_6 + 3 x_7 + 10 x_8 <= 90
constr(6):  10 x_0 + 5 x_1 + 8 x_2 + 2 x_3 + 7 x_4 + 8 x_5 + 6 x_6 + 2 x_7 + 2 x_8 <= 90
constr(7):  7 x_0 + 9 x_1 + x_2 + 9 x_3 + 5 x_4 + 8 x_5 + 5 x_6 + 9 x_7 + 5 x_8 <= 80
constr(8):  2 x_0 + 4 x_1 + 6 x_2 + x_3 + 7 x_4 + 9 x_5 + 10 x_6 + x_7 + 7 x_8 <= 80
constr(9):  8 x_0 + 10 x_1 + 2 x_2 + 6 x_3 + 7 x_4 + 2 x_5 + 2 x_6 + 4 x_7 + 9 x_8 <= 10
constr(10):  10 

In [5]:
branch_binary = BranchAndBoundBinary()
solucao = branch_binary.resolver_modelo('teste4.txt')
branch_binary.save(solucao, 'modelo_4.lp')
print("Função Objetivo: ", solucao.objective_value)
print("Solução: ", [x.x for x in solucao.vars])

\Problem name: 

Minimize
OBJROW: -9 x_0 -7 x_1 -10 x_2 -7 x_3 -9 x_4 -6 x_5 -8 x_6 -4 x_7 -9 x_8
Subject To
constr(0):  4 x_0 + 9 x_1 + 4 x_2 + x_3 + 9 x_4 + 6 x_5 + 3 x_6 + 6 x_7 + x_8 <= 40
constr(1):  3 x_0 + 7 x_1 + 8 x_2 + 7 x_3 + 6 x_4 + 3 x_5 + 5 x_6 + 9 x_7 + 4 x_8 <= 80
constr(2):  9 x_0 + 3 x_1 + 6 x_2 + 5 x_3 + 7 x_4 + x_5 + x_6 + 3 x_7 + 9 x_8 <= 40
constr(3):  5 x_0 + 9 x_1 + 6 x_2 + 5 x_3 + 9 x_4 + 7 x_5 + 8 x_6 + 7 x_7 + 8 x_8 <= 10
constr(4):  7 x_0 + 7 x_1 + 4 x_2 + x_3 + 3 x_4 + 4 x_5 + 8 x_6 + x_7 + 9 x_8 <= 10
constr(5):  x_0 + 6 x_1 + 6 x_2 + x_3 + 6 x_4 + 7 x_5 + 3 x_6 + 8 x_7 + 7 x_8 <= 10
constr(6):  6 x_0 + 6 x_1 + 8 x_2 + 6 x_3 + 10 x_4 + 8 x_5 + x_6 + 4 x_7 + 4 x_8 <= 70
constr(7):  9 x_0 + x_1 + 9 x_2 + 7 x_3 + 10 x_4 + 5 x_5 + 6 x_6 + 2 x_7 + 5 x_8 <= 10
constr(8):  2 x_0 + 7 x_1 + 6 x_2 + 5 x_3 + x_4 + x_5 + 9 x_6 + 2 x_7 + x_8 <= 20
constr(9):  x_2 = 1
constr(10):  x_1 = 0
constr(11):  x_7 = 0
constr(12):  x_8 = 0
constr(13):  x_6 = 0
constr(14):  x_5 = 